# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc4b0e5bb20>
├── 'a' --> tensor([[ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265]])
└── 'x' --> <FastTreeValue 0x7fc4b0e5b430>
    └── 'c' --> tensor([[-1.2668, -1.4606,  0.7595,  0.0697],
                        [-0.2882, -1.3151,  1.6360,  0.0947],
                        [-0.6882, -0.7341,  0.0189, -0.3057]])

In [4]:
t.a

tensor([[ 0.4524, -2.3444,  0.9916],
        [ 0.5875,  1.0119,  0.5265]])

In [5]:
%timeit t.a

76 ns ± 1.03 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc4b0e5bb20>
├── 'a' --> tensor([[-0.5484,  0.8148, -0.9455],
│                   [ 0.9709,  0.2113, -0.2732]])
└── 'x' --> <FastTreeValue 0x7fc4b0e5b430>
    └── 'c' --> tensor([[-1.2668, -1.4606,  0.7595,  0.0697],
                        [-0.2882, -1.3151,  1.6360,  0.0947],
                        [-0.6882, -0.7341,  0.0189, -0.3057]])

In [7]:
%timeit t.a = new_value

79.9 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265]]),
    x: Batch(
           c: tensor([[-1.2668, -1.4606,  0.7595,  0.0697],
                      [-0.2882, -1.3151,  1.6360,  0.0947],
                      [-0.6882, -0.7341,  0.0189, -0.3057]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4524, -2.3444,  0.9916],
        [ 0.5875,  1.0119,  0.5265]])

In [11]:
%timeit b.a

65.7 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.7997, -0.9267, -0.4145],
               [ 1.9690, -1.1728,  0.8529]]),
    x: Batch(
           c: tensor([[-1.2668, -1.4606,  0.7595,  0.0697],
                      [-0.2882, -1.3151,  1.6360,  0.0947],
                      [-0.6882, -0.7341,  0.0189, -0.3057]]),
       ),
)

In [13]:
%timeit b.a = new_value

603 ns ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.04 µs ± 14.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.1 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

283 µs ± 6.66 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

273 µs ± 4.25 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc4844341f0>
├── 'a' --> tensor([[[ 0.4524, -2.3444,  0.9916],
│                    [ 0.5875,  1.0119,  0.5265]],
│           
│                   [[ 0.4524, -2.3444,  0.9916],
│                    [ 0.5875,  1.0119,  0.5265]],
│           
│                   [[ 0.4524, -2.3444,  0.9916],
│                    [ 0.5875,  1.0119,  0.5265]],
│           
│                   [[ 0.4524, -2.3444,  0.9916],
│                    [ 0.5875,  1.0119,  0.5265]],
│           
│                   [[ 0.4524, -2.3444,  0.9916],
│                    [ 0.5875,  1.0119,  0.5265]],
│           
│                   [[ 0.4524, -2.3444,  0.9916],
│                    [ 0.5875,  1.0119,  0.5265]],
│           
│                   [[ 0.4524, -2.3444,  0.9916],
│                    [ 0.5875,  1.0119,  0.5265]],
│           
│                   [[ 0.4524, -2.3444,  0.9916],
│                    [ 0.5875,  1.0119,  0.5265]]])
└── 'x' --> <FastTreeValue 0x7fc4844342e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.4 µs ± 856 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc4b0ecf550>
├── 'a' --> tensor([[ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265],
│                   [ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265],
│                   [ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265],
│                   [ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265],
│                   [ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265],
│                   [ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265],
│                   [ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265],
│                   [ 0.4524, -2.3444,  0.9916],
│                   [ 0.5875,  1.0119,  0.5265]])
└── 'x' --> <FastTreeValue 0x7fc4b0ebaf40>
    └── 'c' --> tensor([[-1.2668, -1.4606,  0.7595,  0.0697],
                        [-0.2882, -1.3151,  1.6360,  0.0947],
                 

In [23]:
%timeit t_cat(trees)

41 µs ± 490 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

81.7 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4524, -2.3444,  0.9916],
                [ 0.5875,  1.0119,  0.5265]],
       
               [[ 0.4524, -2.3444,  0.9916],
                [ 0.5875,  1.0119,  0.5265]],
       
               [[ 0.4524, -2.3444,  0.9916],
                [ 0.5875,  1.0119,  0.5265]],
       
               [[ 0.4524, -2.3444,  0.9916],
                [ 0.5875,  1.0119,  0.5265]],
       
               [[ 0.4524, -2.3444,  0.9916],
                [ 0.5875,  1.0119,  0.5265]],
       
               [[ 0.4524, -2.3444,  0.9916],
                [ 0.5875,  1.0119,  0.5265]],
       
               [[ 0.4524, -2.3444,  0.9916],
                [ 0.5875,  1.0119,  0.5265]],
       
               [[ 0.4524, -2.3444,  0.9916],
                [ 0.5875,  1.0119,  0.5265]]]),
    x: Batch(
           c: tensor([[[-1.2668, -1.4606,  0.7595,  0.0697],
                       [-0.2882, -1.3151,  1.6360,  0.0947],
                       [-0.6882, -0.7341,  0.0189, -0.3057]],
         

In [26]:
%timeit Batch.stack(batches)

105 µs ± 992 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265],
               [ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265],
               [ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265],
               [ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265],
               [ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265],
               [ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265],
               [ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265],
               [ 0.4524, -2.3444,  0.9916],
               [ 0.5875,  1.0119,  0.5265]]),
    x: Batch(
           c: tensor([[-1.2668, -1.4606,  0.7595,  0.0697],
                      [-0.2882, -1.3151,  1.6360,  0.0947],
                      [-0.6882, -0.7341,  0.0189, -0.3057],
                      [-1.2668, -1.4606,  0.7595,  0.0697],
                      [-0.2882, -

In [28]:
%timeit Batch.cat(batches)

187 µs ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

581 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
